##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo crear una canalización de TFX usando plantillas

Nota: Recomendamos ejecutar este tutorial en Google Cloud Vertex AI Workbench. [Inicie este bloc de notas en Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

## Introducción

Este documento proporcionará instrucciones para crear una canalización de TensorFlow Extended (TFX) utilizando las *plantillas* que se proporcionan con el paquete de Python para TFX. Muchas de las instrucciones son comandos de shell de Linux, que se ejecutarán en una instancia de AI Platform Notebooks. Se proporcionan las correspondientes celdas de código de Jupyter Notebook que invocan esos comandos usando `!`.

Compilará una canalización mediante el uso del [conjunto de datos Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) publicado por la ciudad de Chicago. Le recomendamos que use esta canalización como punto de referencia para intentar compilar su propia canalización con su conjunto de datos.


## Paso 1. Configure su entorno

AI Platform Pipelines preparará un entorno de desarrollo para compilar una canalización y un clúster de Kubeflow Pipeline para ejecutar la canalización recién compilada.

**NOTA:** Para seleccionar una versión particular de TensorFlow o seleccionar una instancia de GPU, cree una instancia preinstalada de TensorFlow en AI Platform Notebooks.


Instale el paquete de python para `tfx` con el requisito adicional `kfp`.

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

Revisemos las versiones de TFX.

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

En AI Platform Pipelines, TFX se ejecuta en un entorno de Kubernetes alojado mediante [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/).

Configuremos algunas variables de entorno para usar Kubeflow Pipelines.

Primero, obtenga el ID de su proyecto de GCP.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

También tenemos que acceder a su clúster KFP. Puede acceder a él en su Consola de Google Cloud en el menú "AI Platform &gt; Canalización". El "punto de conexión" del clúster de KFP se puede encontrar en la URL del panel de canalizaciones, o puede obtenerlo en la URL de la página de introducción donde inició este bloc de ntoas. Creemos una variable de entorno `ENDPOINT` y configurémosla en el punto de conexión del clúster KFP. **ENDPOINT debe contener solo la parte del nombre de host de la URL.** Por ejemplo, si la URL del panel de KFP es `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start` , el valor de ENDPOINT se convierte en `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

> **NOTA: A continuación, DEBE establecer el valor de su PUNTO DE CONEXIÓN.**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

Establezca el nombre de la imagen como `tfx-pipeline` en el proyecto GCP actual.

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

Y ya está. Estamos listos para crear una canalización.

## Paso 2. Copie la plantilla predefinida al directorio de su proyecto

En este paso, crearemos un directorio y archivos de proyecto de canalización de trabajo copiando archivos adicionales de una plantilla predefinida.

Puede cambiar el nombre de su canalización si cambia el valor de `PIPELINE_NAME` a continuación. Este también se convertirá en el nombre del directorio del proyecto donde se colocarán sus archivos.

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

TFX incluye la plantilla <code>taxi</code> con el paquete de Python para TFX. Si planea resolver un problema de predicción puntual, que incluye clasificación y regresión, esta plantilla podría usarse como punto de partida.

El comando CLI `tfx template copy` copia archivos de plantilla predefinidos al directorio de su proyecto.

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

Cambie el contexto del directorio de trabajo en este bloc de notas al directorio del proyecto.

In [ ]:
%cd {PROJECT_DIR}

> NOTA: Recuerde que debe cambiar el directorio en `File Browser` a la izquierda haciendo clic en el directorio del proyecto una vez que lo haya creado.

## Paso 3. Explore sus archivos fuente copiados

La plantilla de TFX proporciona archivos de estructura básicos para compilar una canalización, incluido el código fuente de Python, datos de muestra y bloc de notas Jupyter para analizar la salida de la canalización. La plantilla `taxi` utiliza el mismo conjunto de datos *Chicago Taxi* y el mismo modelo de ML que el [Tutorial de Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

Esta es una breve introducción a cada uno de los archivos de Python.

- `pipeline`: este directorio contiene la definición de la canalización.
    - `configs.py`: define constantes comunes para los ejecutores de la canalización
    - `pipeline.py`: define los componentes de TFX y una canalización
- `models`: este directorio contiene definiciones de modelos de ML.
    - `features.py`, `features_test.py`: define características para el modelo
    - `preprocessing.py`, `preprocessing_test.py`: define trabajos de preprocesamiento con ayuda de `tf::Transform`
    - `estimator`: este directorio contiene un modelo basado en Estimator.
        - `constants.py`: define las constantes del modelo
        - `model.py`, `model_test.py`: define el modelo DNN utilizando el estimador TF
    - `keras`: este directorio contiene un modelo basado en Keras.
        - `constants.py`: define las constantes del modelo
        - `model.py`, `model_test.py`: define el modelo DNN usando Keras
- `local_runner.py`, `kubeflow_runner.py`: define ejecutores para cada motor de orquestación


Quizás note que hay algunos archivos con `_test.py` en su nombre. Estas son pruebas unitarias de la canalización y se recomienda agregar más pruebas unitarias a medida que implemente sus propias canalizaciones. Puede ejecutar pruebas unitarias si proporciona el nombre del módulo de los archivos de prueba con la marca `-m`. Generalmente puede obtener el nombre de un módulo al eliminar la extensión `.py` y reemplazarla por `/` con `.`. Por ejemplo:

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## Paso 4. Ejecute su primera canalización de TFX

Los componentes en la canalización de TFX generarán salidas para cada ejecución como [artefactos de metadatos de ML](https://www.tensorflow.org/tfx/guide/mlmd) y deben almacenarse en algún lugar. Puede utilizar cualquier almacenamiento al que pueda acceder el clúster de KFP y, para este ejemplo, utilizaremos Google Cloud Storage (GCS). Se debería haber creado automáticamente un depósito de GCS predeterminado. Su nombre será `<your-project-id>-kubeflowpipelines-default`.


Carguemos nuestros datos de muestra en el depósito de GCS para que podamos usarlos en nuestra canalización más adelante.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

Creemos una canalización de TFX a partir del comando `tfx pipeline create`.

> Nota: Al crear una canalización para KFP, necesitamos una imagen de contenedor que se utilizará para ejecutar nuestra canalización. Y `skaffold` compilará la imagen para nosotros. Debido a que skaffold extrae imágenes base del Docker Hub, tomará entre 5 y 10 minutos cuando compilemos la imagen por primera vez, pero tomará mucho menos tiempo desde la segunda compilación.

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

Mientras se crea una canalización, se generará `Dockerfile` para compilar una imagen de Docker. No olvide agregarlo al sistema de control de código fuente (por ejemplo, git) junto con otros archivos fuente.

NOTA: `kubeflow` se seleccionará automáticamente como motor de orquestación si `airflow` no está instalado y no se especifica `--engine`.

Ahora inicie una ejecución con la canalización recién creada con ayuda del comando `tfx run create`.

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

O también puede ejecutar la canalización en el panel de KFP. La nueva ejecución aparecerá en Experimentos en el Panel de de KFP. Al hacer clic en el experimento, podrá monitorear el progreso y visualizar los artefactos creados durante la ejecución.

Sin embargo, recomendamos visitar el Panel de KFP. Puede acceder al panel de KFP desde el menú de Cloud AI Platform Pipelines en la Consola de Google Cloud. Una vez que visite el panel, podrá encontrar la canalización y acceder a una gran cantidad de información sobre esta. Por ejemplo, puede encontrar sus ejecuciones en el menú *Experimentos* y, cuando abre su ejecución en Experimentos, puede encontrar todos sus artefactos de la canalización en el menú *Artefactos*.

> Nota: Si la ejecución de su canalización falla, puede ver registros detallados para cada componente de TFX en la pestaña Experimentos en el Panel de KFP.

Una de las principales fuentes de error son los problemas relacionados con los permisos. Asegúrese de que su clúster de KFP tenga permisos para acceder a las API de Google Cloud. Esto se puede configurar [al momento de crear un clúster de KFP en GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) o bien puede consultar el [documento de solución de problemas en GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## Paso 5. Agregue componentes para la validación de datos

En este paso, agregará componentes para la validación de datos, incluidos `StatisticsGen`, `SchemaGen` y `ExampleValidator`. Si está interesado en la validación de datos, consulte [Introducción a Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

> **Haga doble clic para cambiar el directorio a `pipeline` y haga doble clic nuevamente para abrir `pipeline.py`**. Busque y descomente las 3 líneas que agregan `StatisticsGen`, `SchemaGen` y `ExampleValidator` a la canalización. (Consejo: busque comentarios que contengan `TODO(step 5):` :). Asegúrese de guardar `pipeline.py` después de editarlo.

Ahora debe actualizar la canalización existente con la definición de canalización modificada. Utilice el comando `tfx pipeline update` para actualizar su canalización, seguido del comando `tfx run create` para crear una nueva ejecución de su canalización actualizada.


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### Verifique las salidas de la canalización

Visite el panel de KFP para buscar salidas de canalización en la página para la ejecución de su canalización. Haga clic en la pestaña "Experimentos" a la izquierda y en "Todas las ejecuciones" en la página Experimentos. Debería poder encontrar la ejecución con el nombre de su canalización.

## Paso 6. Agregue componentes para el entrenamiento

En este paso, agregará componentes para el entrenamiento y la validación del modelo, incluidos `Transform`, `Trainer`, `Resolver`, `Evaluator` y `Pusher`.

> **Haga doble clic para abrir `pipeline.py`**. Busque y descomente las 5 líneas que agregan `Transform`, `Trainer`, `Resolver`, `Evaluator` y `Pusher` a la canalización. (Consejo: busque `TODO(step 6):`

Como hizo antes, ahora necesita actualizar la canalización existente con la definición de canalización modificada. Las instrucciones son las mismas que las del Paso 5. Actualice la canalización con ayuda de `tfx pipeline update` y use `tfx run create` para crear una ejecución.


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Cuando esta ejecución finalice correctamente, habrá creado y ejecutado su primera canalización de TFX en AI Platform Pipelines.

**NOTA:** Si cambiamos algo en el código del modelo, también tenemos que volver a compilar la imagen del contenedor. Podemos activar la reconstrucción usando la marca `--build-image` en el comando `pipeline update`.

**NOTA:** Es posible que haya notado que cada vez que creamos una ejecución de canalización, cada componente se ejecuta una y otra vez aunque la entrada y los parámetros no hayan cambiado. Es una pérdida de tiempo y recursos, y puede omitir esas ejecuciones con el almacenamiento en caché de la canalización. Puede habilitar el almacenamiento en caché si especifica `enable_cache=True` para el objeto `Pipeline` en `pipeline.py`.


## Paso 7. (*Opcional*) Pruebe BigQueryExampleGen

[BigQuery](https://cloud.google.com/bigquery) es un almacén de datos en la nube sin servidor, altamente escalable y rentable. BigQuery sirve como fuente de ejemplos de entrenamiento en TFX. En este paso, agregaremos `BigQueryExampleGen` a la canalización.

> **Haga doble clic para abrir `pipeline.py`**. Comente `CsvExampleGen` y descomente la línea que crea una instancia de `BigQueryExampleGen`. También se debe descomentar el argumento `query` de la función `create_pipeline`.

Tenemos que especificar qué proyecto de GCP se usará para BigQuery, y para hacer esto hay que configurar `--project` en `beam_pipeline_args` cuando se crea una canalización.

> **Haga doble clic para abrir `configs.py`**. Descomente la definición de `GOOGLE_CLOUD_REGION`, `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` y `BIG_QUERY_QUERY`. Debe reemplazar el valor de la región en este archivo con los valores correctos para su proyecto de GCP.

> **Nota: DEBE configurar su región GCP en el archivo `configs.py` antes de continuar.**

> **Cambie el directorio un nivel hacia arriba.** Haga clic en el nombre del directorio encima de la lista de archivos. El nombre del directorio es el nombre de la canalización, que es `my_pipeline` si no lo cambió.

> **Haga doble clic para abrir `kubeflow_runner.py`**. Descomente dos argumentos, `query` y `beam_pipeline_args`, para la función `create_pipeline`.

Ahora la canalización está lista para usar BigQuery como fuente de ejemplo. Actualice la canalización como antes y cree una nueva ejecución como lo hicimos en los pasos 5 y 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## Paso 8. (*Opcional*) Pruebe Dataflow con KFP

Varios [componentes de TFX usan Apache Beam](https://www.tensorflow.org/tfx/guide/beam) para implementar canalizaciones de datos paralelas, lo que significa que puede distribuir cargas de trabajo de procesamiento de datos mediante [Google Cloud Dataflow](https://cloud.google.com/dataflow/). En este paso, configuraremos el orquestador de Kubeflow para que use Dataflow como backend de procesamiento de datos para Apache Beam.

> **Haga doble clic en `pipeline` para cambiar de directorio y haga doble clic para abrir `configs.py`**. Descomente la definición de `GOOGLE_CLOUD_REGION` y `DATAFLOW_BEAM_PIPELINE_ARGS`.

> **Cambie el directorio un nivel hacia arriba.** Haga clic en el nombre del directorio encima de la lista de archivos. El nombre del directorio es el nombre de la canalización, que es `my_pipeline` si no lo cambió.

> **Haga doble clic para abrir `kubeflow_runner.py`**. Descomente `beam_pipeline_args`. (También asegúrese de comentar `beam_pipeline_args` actuales que agregó en el Paso 7).

Ahora la canalización está lista para usar Dataflow. Actualice la canalización y cree una ejecución como lo hicimos en los pasos 5 y 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Puede encontrar sus trabajos de Dataflow en [Dataflow en la consola de Cloud](http://console.cloud.google.com/dataflow).


## Paso 9. (*Opcional*) Pruebe entrenamiento y predicción de Cloud AI Platform con KFP

TFX interopera con varios servicios administrados de GCP, como [Cloud AI Platform Training y Prediction](https://cloud.google.com/ai-platform/). Puede configurar su componente `Trainer` para que use Cloud AI Platform Training, un servicio administrado para entrenar modelos de ML. Además, cuando su modelo esté compilado y listo para ser servido, puede *insertarlo* en Cloud AI Platform Prediction para su servicio. En este paso, configuraremos nuestro componente `Trainer` y `Pusher` para utilizar los servicios de Cloud AI Platform.

> Antes de editar archivos, es posible que primero tenga que habilitar la *API de Training y Prediction de AI Platform*.

> **Haga doble clic en `pipeline` para cambiar de directorio y haga doble clic para abrir `configs.py`**. Descomente la definición de `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` y `GCP_AI_PLATFORM_SERVING_ARGS`. Usaremos nuestra imagen de contenedor personalizada para entrenar un modelo en Cloud AI Platform Training, por lo que debemos configurar `masterConfig.imageUri` en `GCP_AI_PLATFORM_TRAINING_ARGS` con el mismo valor que `CUSTOM_TFX_IMAGE` arriba.

> **Cambie el directorio un nivel hacia arriba y haga doble clic para abrir `kubeflow_runner.py`**. Descomente `ai_platform_training_args` y `ai_platform_serving_args`.

Actualice la canalización y cree una ejecución como lo hicimos en los pasos 5 y 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Puede encontrar sus trabajos de entrenamiento en [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). Si su canalización se completó correctamente, puede encontrar su modelo en [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## Paso 10. Ingiera SUS datos en la canalización

Creamos una canalización para un modelo utilizando el conjunto de datos Chicago Taxi. Ahora es el momento de poner sus datos en proceso.

Sus datos se pueden almacenar en cualquier lugar al que pueda acceder su canalización, incluido GCS o BigQuery. Deberá modificar la definición de la canalización para acceder a sus datos.

1. Si sus datos están almacenados en archivos, modifique `DATA_PATH` en `kubeflow_runner.py` o `local_runner.py` y configúrelo en la ubicación de sus archivos. Si sus datos están almacenados en BigQuery, modifique `BIG_QUERY_QUERY` en `pipeline/configs.py` para consultar correctamente sus datos.
2. Agregue características en `models` / <code>features.py</code>.
3. Modifique `models` / <code>preprocessing.py</code> para <a>transformar los datos de entrada para el entrenamiento</a>.
4. Modifique `models` / `keras` / <code>model.py</code> y <code>models</code> / <code>keras</code> / <code>constants.py</code> para <a>describir su modelo de ML</a>.
    - También puede utilizar un modelo basado en un estimador. Cambie la constante `RUN_FN` a `models.estimator.model.run_fn` en `pipeline/configs.py`.

Consulte la [guía del componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para obtener más información.

## Limpieza

Para limpiar todos los recursos de Google Cloud utilizados en este proyecto, puede [eliminar el proyecto de Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) que se usó para el tutorial.

Alternativamente, puede limpiar recursos individuales si visita cada consola:

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
